In [1]:
using Random
using Distributions
#using Plots

Random.seed!(1234);
#gr();

In [2]:
include("../distribution/bernoulli.jl");
using .bernoulli_dist

## Bernoulli Distributions

$$
p(x | \mu) = x^{\mu} \cdot (1 - x)^{1 - \mu}
$$

In [4]:
mu = [0.0, 1.0, 1.0, 1.0];
bernoulli = BernoulliDist(mu);
bernoulli_dist.pdf(bernoulli, 0)

0.25